In [1]:
import pandas as pd
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import networkx as nx
from scipy.sparse import lil_matrix
import matplotlib.pyplot as plt

from utils import*

In [2]:
data = pd.read_csv('lenta-ru-train.csv')

In [3]:
data

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1
...,...,...,...,...
206632,Бывшую вице-президента Tiffany обвинили в милл...,Бывшей вице-президенту ювелирной компании Tiff...,Экономика,0
206633,Минфин предложил увеличить штрафы за незаконны...,Минфин предлагает увеличить размер штрафа для ...,Экономика,0
206634,Потребительская уверенность россиян вернулась ...,Индекс потребительской уверенности россиян в п...,Экономика,0
206635,Тарпищеву дали американскую визу,Капитан женской сборной России по теннису Шами...,Спорт,1


### examples of different graph metrics to compute distance between texts

In [4]:
test = data[0:100]

In [5]:
graph_1 = text2graph(test['text'][0], 0.1)
graph_2 = text2graph(test['text'][1], 0.1)

In [6]:
jaccard_similarity(graph_1, graph_2) # diff between texts of economics and culture topics

0.017

In [7]:
graph_3 = text2graph(test['text'][7], 0.1)
graph_4 = text2graph(test['text'][9], 0.1)

In [8]:
jaccard_similarity(graph_1, graph_3) # diff between texts of economics and economics topics

0.105

In [9]:
nx.average_clustering(graph_1) - nx.average_clustering(graph_2)

0.11201891071746739

### ok, lets do some search engine

In [10]:
%%time
test['dist'] = test['text'].apply(lambda x: nx.average_clustering(text2graph(x, 0.1)))

CPU times: user 47.4 s, sys: 28.7 ms, total: 47.4 s
Wall time: 47.5 s


/home/mitya/PycharmProjects/nlp_graph/nlp_graph/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
#test['dist_jaccard'] = 

In [12]:
query = 'Художница купила консоль PlayStation 5 в интернете, нарвалась на мошенника и отомстила ему. Соответствующий материал публикует портал Bored Panda.Американка Бриттани Эверет (Brittany Everette) рассказала, что ей предложили приобрести бездисковую Playstation 5 за цену, по которой ее продают в магазинах, а именно, 450 долларов (34 тысячи рублей). Мужчина объяснил, что его девушка подарила ему не ту версию приставки.'

In [13]:
preprocess_query(query, nx.average_clustering, 0.1)

0.6321573989018345

In [14]:
# results may vary because of text2graph algorithm
%time
find_closest(query, nx.average_clustering, test, 0.1)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.2 µs


/home/mitya/PycharmProjects/nlp_graph/utils.py:254: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'] = df['dist'].apply(lambda x: abs(query_preproc - x))


63    Суд в Гааге наложил временный запрет на продаж...
73    Международная федерация звукозаписывающей инду...
53    Американский писатель Дэн Браун (Dan Brown), а...
74    В пятницу основатели студии Miramax братья Хар...
11    Российскому боксеру Федору Чудинову вернут тит...
                            ...                        
81    Наиболее доходной профессией в настоящее время...
45    Гендиректор «Почты России» Дмитрий Страшнов уд...
9     Чистый приток вкладов населения в российские б...
0     Глава Росстата Владимир Соколин заявил, что в ...
71    Альфа-банк на следующей неделе может подать ис...
Name: text, Length: 100, dtype: object

#### average distance between textgraphs inside topics

In [15]:
print(average_distance.__doc__)


    computes average distance by the chosen metric between text-graphs of certain topic.

    arguments:
    df -- dataframe to analyze
    topic -- class of texts to choose
    column -- column to analyze
    func -- metric to use

    


In [16]:
average_distance(test, 'Экономика', 'text', jaccard_similarity)

0.088

In [17]:
average_distance(test, 'Культура', 'text', jaccard_similarity)

0.069

In [18]:
average_distance(test, 'Бизнес', 'text', nx.average_clustering)

0.737

In [19]:
#average_distance(test, 'Экономика', nx.average_clustering) # division by zero

In [20]:
# ideas:
# 1.make 'average' graph for each class
#   make function to diff them

# 2.graph representation of texts --> vector representation
#   distance metric vector representations

# 3.graph distance metric

# 4.compute average distance between graphs inside topics